## OPTIMIZATION

!pip install tensorflow


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Value counts for Status
# Since reult of STATUS count is that all but 5 have a Status of 1, we could possibly drop it.
application_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [3]:
# Value counts for SPECIAL_CONSIDERATIONS
# Since reult of SPECIAL_CONSIDERATIONS count is that only 27 have a special consideration, good candidate to drop.
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [4]:
# Status and Special Considerations can both be dropped along with EIN during this interative attempt 
# to drop the modelbecause they are small percentage or number that don't have a status of 1 and 
# only 27 have Y as a special consideration.

In [5]:
# For the optimization model, we are going to just drop the EIN but leave the Name to see if it has any affect on the model
# Honestly, I am skeptical about the usefulness of value of keeping the name in the dataset.
# If it does in and of itself have an impact in increating accuracy, it will cause me to have less confidence in the model
# Unless I cqn determine a cause or "the why"
# According to the video shared by Dr A, the name may be retained in case it is repeated.
# Note:Special considerations and Statuses were only two unique values -- perhaps not have to use them as much

# Summary: Dropping EIN, STATUS and SPECISL CONSIDERATIONS as non beneficial columns.

application_df = application_df.drop(["EIN"], 1)
application_df = application_df.drop(["STATUS"], 1)
application_df = application_df.drop(["SPECIAL_CONSIDERATIONS"], 1)
application_df.head(5)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [6]:
application_df.head(10)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
5,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,Independent,C1200,Preservation,Trust,0,5000,1
6,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,Independent,C1000,Preservation,Trust,100000-499999,31452,1
7,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,10M-50M,7508025,1
8,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1-9999,94389,1
9,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0


INPUTS-APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, INCOME_AMT, ASK_AMT

OUTPUTS-IS_SUCCESSSFUL-0 if not successful, 1 if successful

In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [8]:
# Look at Name Value value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts

# How many name counts are greater than 5?
name_counts[name_counts > 5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
THE READING FOUNDATION                                6
THE INTERIOR DESIGN SOCIETY INC                       6
AMERICAN INDEPENDENT BUSINESS ALLIANCE                6
QUAIL FEDERATION INC                                  6
PUBLIC RELATIONS COUNCIL OF ALABAMA                   6
Name: NAME, Length: 354, dtype: int64

Suggested name counts <= 5 to be combined into "other"

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `names_to_replace`
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for app in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                      20043
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
PUBLIC RELATIONS COUNCIL OF ALABAMA            6
TEXAS FEDERATION OF WOMEN CLUBS                6
DELTA KAPPA GAMMA SOCIETY INTERNATIONAL        6
SOCIETY FOR CREATIVE ANACHRONISM               6
THE INTERIOR DESIGN SOCIETY INC                6
Name: NAME, Length: 355, dtype: int64

In [10]:
# Just realized the reason we are keeping name in this activity -- the succcess of the campaign or organization may be due
# in part to the name sake or name recognition of the entity.  It could be as basic as the name recognition assists or plays
# a factor in funding being received.

In [11]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C1570        1
C2500        1
C2190        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C4100        6
C1720        6
C1257        5
C1600        5
C2710        3
C0           3
C1260        3
C1256        2
C3200        2
C1234        2
C1267        2
C1246        2
Name: CLASSIFICATION, dtype: int64

Updates suggested cutoff is 1000 for the calssification counts

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df)
application_with_dummies_df.head(5)

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [18]:
# Split our preprocessed data into our features and target arrays
x = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

STEP 2: Compile,Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(x_train[0])
layer1Nodes = 100
layer2Nodes = 30
layer3Nodes = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1Nodes, input_dim=number_input_features, activation='relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer2Nodes, input_dim=layer1Nodes, activation='sigmoid')
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer3Nodes, input_dim=layer1Nodes, activation='sigmoid')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               39600     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 42,951
Trainable params: 42,951
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(x_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 992us/step - loss: 0.5160 - accuracy: 0.7650
Epoch 2/100
804/804 [==============================] - 1s 990us/step - loss: 0.4471 - accuracy: 0.7918
Epoch 3/100
804/804 [==============================] - 1s 979us/step - loss: 0.4355 - accuracy: 0.7944
Epoch 4/100
804/804 [==============================] - 1s 978us/step - loss: 0.4297 - accuracy: 0.7968
Epoch 5/100
804/804 [==============================] - 1s 984us/step - loss: 0.4266 - accuracy: 0.7976
Epoch 6/100
804/804 [==============================] - 1s 981us/step - loss: 0.4248 - accuracy: 0.7982
Epoch 7/100
804/804 [==============================] - 1s 981us/step - loss: 0.4224 - accuracy: 0.7990
Epoch 8/100
804/804 [==============================] - 1s 987us/step - loss: 0.4215 - accuracy: 0.7989
Epoch 9/100
804/804 [==============================] - 1s 987us/step - loss: 0.4201 - accuracy: 0.7994
Epoch 10/100
804/804 [==============================] - 1s 982us/step - l

804/804 [==============================] - 1s 1000us/step - loss: 0.4009 - accuracy: 0.8063
Epoch 81/100
804/804 [==============================] - 1s 984us/step - loss: 0.4015 - accuracy: 0.8084
Epoch 82/100
804/804 [==============================] - 1s 978us/step - loss: 0.4006 - accuracy: 0.8073
Epoch 83/100
804/804 [==============================] - 1s 985us/step - loss: 0.4007 - accuracy: 0.8080
Epoch 84/100
804/804 [==============================] - 1s 983us/step - loss: 0.4001 - accuracy: 0.8086
Epoch 85/100
804/804 [==============================] - 1s 983us/step - loss: 0.4009 - accuracy: 0.8080
Epoch 86/100
804/804 [==============================] - 1s 976us/step - loss: 0.4008 - accuracy: 0.8076
Epoch 87/100
804/804 [==============================] - 1s 983us/step - loss: 0.4007 - accuracy: 0.8079
Epoch 88/100
804/804 [==============================] - 1s 993us/step - loss: 0.4011 - accuracy: 0.8079
Epoch 89/100
804/804 [==============================] - 1s 986us/step - loss

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4622 - accuracy: 0.7907
Loss: 0.4622493386268616, Accuracy: 0.7906705737113953


In [26]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity-OptimizeModel.h5")

CONCLUSION: Accuracy for the opti,ized model is 79.07% and loss was 40.02%  With this model we are/were able to predict approximately 79% of the campaigns or organizations that would be successful. We achieved a target predictive accuracy higher than 75% by dropping the "STATUS" and "SPECIAL CONSIDERATIONS" Columns.  However, we retained the "NAME" column in case name sake or reputation were part of the success of the campaign.  Several changes were made to the model: 1) Changed the layer1Nodes to 100  2) Added a third set of nodes with a value of 30.  Thus, we had three hidden nodes with node values for 100, 30 and 10, respectively; 3) The activation for the second hidden layer was change from 'relu" to 'sigmoid'.